# Mate-in-One Position Comparison: Original vs Flipped

This notebook visualizes mate-in-one positions and their color-flipped counterparts side by side.
Each row shows the original position (left) and the flipped position (right) where colors and turns are swapped.


In [17]:
import json
import chess
import chess.svg
from IPython.display import SVG, HTML, display
from pathlib import Path
import sys

# Add project root to path
project_root = Path.cwd().parent if Path.cwd().name == 'test' else Path.cwd()
sys.path.insert(0, str(project_root))


In [18]:
def rotate_square_180(square):
    """Rotate a square 180 degrees: a1->h8, h8->a1, etc."""
    return 63 - square

def flip_fen(fen):
    """Flip FEN by rotating board 180 degrees (vertically and horizontally) AND swapping colors."""
    try:
        board = chess.Board(fen)
        
        # Create a new empty board
        flipped_board = chess.Board()
        flipped_board.clear()
        
        # Rotate 180 degrees AND swap colors
        for square in chess.SQUARES:
            piece = board.piece_at(square)
            if piece:
                # Calculate rotated square (180 degrees)
                rotated_square = rotate_square_180(square)
                
                # Swap color: WHITE -> BLACK, BLACK -> WHITE
                flipped_color = chess.BLACK if piece.color == chess.WHITE else chess.WHITE
                flipped_piece = chess.Piece(piece.piece_type, flipped_color)
                flipped_board.set_piece_at(rotated_square, flipped_piece)
        
        # Swap the turn
        flipped_board.turn = chess.BLACK if board.turn == chess.WHITE else chess.WHITE
        
        # Rotate and swap castling rights
        flipped_castling = chess.BB_EMPTY
        if board.has_kingside_castling_rights(chess.WHITE):
            # White kingside (h1) -> Black queenside (a8) after 180 rotation
            flipped_castling |= chess.BB_FILE_A & chess.BB_RANK_8
        if board.has_queenside_castling_rights(chess.WHITE):
            # White queenside (a1) -> Black kingside (h8) after 180 rotation
            flipped_castling |= chess.BB_FILE_H & chess.BB_RANK_8
        if board.has_kingside_castling_rights(chess.BLACK):
            # Black kingside (h8) -> White queenside (a1) after 180 rotation
            flipped_castling |= chess.BB_FILE_A & chess.BB_RANK_1
        if board.has_queenside_castling_rights(chess.BLACK):
            # Black queenside (a8) -> White kingside (h1) after 180 rotation
            flipped_castling |= chess.BB_FILE_H & chess.BB_RANK_1
        
        flipped_board.castling_rights = flipped_castling
        
        # Rotate en passant square (if any)
        if board.ep_square is not None:
            rotated_ep = rotate_square_180(board.ep_square)
            flipped_board.ep_square = rotated_ep
        
        # Swap the move counters (flip perspective completely)
        flipped_board.halfmove_clock = board.fullmove_number
        flipped_board.fullmove_number = board.halfmove_clock
        
        return flipped_board.fen()
    except Exception as e:
        print(f"Error flipping FEN: {e}")
        return None

# Check if doubled dataset exists, otherwise create it
doubled_path = project_root / "config" / "mate_in_one_positions_doubled.json"
original_path = project_root / "config" / "mate_in_one_positions.json"

if doubled_path.exists():
    print("Loading doubled dataset...")
    with open(doubled_path, 'r') as f:
        doubled_data = json.load(f)
    print(f"Loaded {len(doubled_data)} positions from doubled dataset")
    
    # Extract pairs for visualization
    with open(original_path, 'r') as f:
        original_data = json.load(f)
    
    comparison_pairs = []
    for fen in list(original_data.keys())[:50]:  # First 50 for visualization
        flipped = flip_fen(fen)
        if flipped and flipped in doubled_data:
            comparison_pairs.append({
                'original': fen,
                'flipped': flipped,
                'weight': doubled_data[fen]['weight']
            })
else:
    print("Creating doubled dataset...")
    # Load original positions
    with open(original_path, 'r') as f:
        original_data = json.load(f)
    
    print(f"Loaded {len(original_data)} original positions")
    
    # Create doubled dataset
    doubled_data = {}
    for fen, config in original_data.items():
        doubled_data[fen] = config.copy()
        flipped = flip_fen(fen)
        if flipped:
            doubled_data[flipped] = {
                'weight': config['weight'],
                'quality': config['quality']
            }
    
    # Recalculate weights
    total_positions = len(doubled_data)
    equal_weight = 1.0 / total_positions
    for fen in doubled_data:
        doubled_data[fen]['weight'] = equal_weight
    
    # Save doubled dataset
    with open(doubled_path, 'w') as f:
        json.dump(doubled_data, f, indent=2)
    
    print(f"Created doubled dataset with {len(doubled_data)} positions")
    print(f"New weight per position: {equal_weight:.15f}")
    
    # Create comparison pairs for visualization
    comparison_pairs = []
    for fen in list(original_data.keys())[:50]:
        flipped = flip_fen(fen)
        if flipped:
            comparison_pairs.append({
                'original': fen,
                'flipped': flipped,
                'weight': equal_weight
            })

print(f"\nCreated {len(comparison_pairs)} comparison pairs for visualization")


Loading doubled dataset...
Loaded 10006 positions from doubled dataset

Created 50 comparison pairs for visualization


In [19]:
def create_comparison_row(orig_fen, flip_fen, pair_num):
    """Create HTML row showing two boards side by side."""
    try:
        orig_board = chess.Board(orig_fen)
        flip_board = chess.Board(flip_fen)
        
        orig_turn = "White" if orig_board.turn == chess.WHITE else "Black"
        flip_turn = "White" if flip_board.turn == chess.WHITE else "Black"
        
        # Create SVG for both boards
        orig_svg = chess.svg.board(orig_board, size=300)
        flip_svg = chess.svg.board(flip_board, size=300)
        
        # Create HTML row with black text
        html = f"""
        <div style="display: flex; gap: 20px; margin: 20px 0; padding: 15px; border: 1px solid #ddd; border-radius: 8px; background: #f9f9f9;">
            <div style="flex: 1; text-align: center; color: #000000;">
                <h4 style="margin-top: 0; color: #000000;">Original #{pair_num} - {orig_turn} to move</h4>
                {orig_svg}
                <div style="margin-top: 10px; font-family: monospace; font-size: 11px; word-break: break-all; background: white; padding: 8px; border-radius: 4px; color: #000000;">
                    {orig_fen}
                </div>
            </div>
            <div style="flex: 1; text-align: center; color: #000000;">
                <h4 style="margin-top: 0; color: #000000;">Flipped #{pair_num} - {flip_turn} to move</h4>
                {flip_svg}
                <div style="margin-top: 10px; font-family: monospace; font-size: 11px; word-break: break-all; background: white; padding: 8px; border-radius: 4px; color: #000000;">
                    {flip_fen}
                </div>
            </div>
        </div>
        """
        return HTML(html)
    except Exception as e:
        return HTML(f"<p style='color: #000000;'>Error displaying pair {pair_num}: {e}</p>")


## Position Comparisons

Each row shows the original position (left) and its color-flipped counterpart (right).


In [20]:
# Display all comparison pairs
for idx, pair in enumerate(comparison_pairs, 1):
    display(create_comparison_row(pair['original'], pair['flipped'], idx))
